In [ ]:
from google.colab import drive
drive.mount('/content/drive')



In [ ]:
!pip install numpy
!pip install keras
!pip install scikit-learn
!pip install matplotlib
!pip install tensorflow


In [ ]:
!curl -O https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/cuda-ubuntu1804.pin
!sudo mv cuda-ubuntu1804.pin /etc/apt/preferences.d/cuda-repository-pin-600
!wget https://developer.download.nvidia.com/compute/cuda/11.4.1/local_installers/cuda-repo-ubuntu1804-11-4-local_11.4.1-470.57.02-1_amd64.deb
!sudo dpkg -i cuda-repo-ubuntu1804-11-4-local_11.4.1-470.57.02-1_amd64.deb
!sudo apt-key add /var/cuda-repo-ubuntu1804-11-4-local/7fa2af80.pub
!sudo apt-get update


In [ ]:
import torch

if torch.cuda.is_available():
    print('CUDA is available!')
    print('Device name:', torch.cuda.get_device_name(0))
else:
    print('CUDA is not available.')


In [ ]:
import os
import numpy as np
import random
from tensorflow.keras.utils import load_img, img_to_array
from PIL import Image

from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

In [ ]:
img_width, img_height = 664, 947

data_folder_fake = '/content/drive/MyDrive/resimler/resimler/fake'
data_folder_reel = '/content/drive/MyDrive/resimler/resimler/real'

fake_images = os.listdir(data_folder_fake)
real_images = os.listdir(data_folder_reel)
type(fake_images)

print(fake_images)

In [ ]:
fake_images

# Define the dimensions of your images
img_width, img_height = 664, 947

data_folder_fake = '/content/drive/MyDrive/image/images/fake'
data_folder_reel = '/content/drive/MyDrive/image/chickies/real'

fake_images = os.listdir(data_folder_fake)
real_images = os.listdir(data_folder_reel)

# Shuffle the images
random.shuffle(fake_images)
random.shuffle(real_images)

# Divide the data into train, validation, and test sets
train_split = 0.6
val_split = 0.2
test_split = 0.2

fake_train = fake_images[:int(train_split * len(fake_images))]
fake_val = fake_images[int(train_split * len(fake_images)):int((train_split + val_split) * len(fake_images))]
fake_test = fake_images[int((train_split + val_split) * len(fake_images)):]

real_train = real_images[:int(train_split * len(real_images))]
real_val = real_images[int(train_split * len(real_images)):int((train_split + val_split) * len(real_images))]
real_test = real_images[int((train_split + val_split) * len(real_images)):]

# Concatenate the train, validation, and test sets
train_images = fake_train + real_train
val_images = fake_val + real_val
test_images = fake_test + real_test

# Shuffle the train, validation, and test sets
random.shuffle(train_images)
random.shuffle(val_images)
random.shuffle(test_images)

In [ ]:
train_data = []
train_labels = []
img_width, img_height = 664 , 947

for img_name in train_images:
 
    if img_name in fake_train:
        label = 1
        img_path = os.path.join(data_folder_fake, img_name)
        img = Image.open(img_path)
        rgb_img = img.convert('RGB')
        rgb_img.save(img_path, format='JPEG', quality=50)
    else:
        label = 0
        img_path = os.path.join(data_folder_reel, img_name)
        img = Image.open(img_path)
        rgb_img = img.convert('RGB')
        rgb_img.save(img_path, format='JPEG', quality=50)

    img = load_img(img_path, target_size=(img_height, img_width))
    img_array = img_to_array(img)
    train_data.append(img_array)
    train_labels.append(label)

train_data = np.array(train_data)
train_labels = np.array(train_labels)

# Normalize the data
train_data = train_data / 255.0

# Convert labels to one-hot encoding
train_labels = to_categorical(train_labels)

In [ ]:
val_data = []
val_labels = []

for img_name in val_images:
    if img_name in fake_val:
        label = 1
        img_path = os.path.join(data_folder_fake, img_name)
        img = Image.open(img_path)
        rgb_img = img.convert('RGB')
        rgb_img.save(img_path, format='JPEG', quality=50)
    else:
        label = 0
        img_path = os.path.join(data_folder_reel, img_name)
        img = Image.open(img_path)
        rgb_img = img.convert('RGB')
        rgb_img.save(img_path, format='JPEG', quality=50)

    img = load_img(img_path, target_size=(img_height, img_width))
    img_array = img_to_array(img)
    val_data.append(img_array)
    val_labels.append(label)

val_data = np.array(val_data)
val_labels = np.array(val_labels)

# Normalize the data
val_data = val_data / 255.0

# Convert labels to one-hot encoding
val_labels = to_categorical(val_labels)

In [ ]:
test_data = []
test_labels = []

for img_name in test_images:
    if img_name in fake_test:
        label = 1
        img_path = os.path.join(data_folder_fake, img_name)
        img = Image.open(img_path)
        rgb_img = img.convert('RGB')
        rgb_img.save(img_path, format='JPEG', quality=50)
    else:
        label = 0
        img_path = os.path.join(data_folder_reel, img_name)
        img = Image.open(img_path)
        rgb_img = img.convert('RGB')
        rgb_img.save(img_path, format='JPEG', quality=50)

    img = load_img(img_path, target_size=(664 , 947))
    img_array = img_to_array(img)
    test_data.append(img_array)
    test_labels.append(label)

test_data = np.array(test_data)
test_labels = np.array(test_labels)


test_data = test_data / 255.0

# one-hot encoding
test_labels = to_categorical(test_labels)

In [ ]:
model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=( 947,664, 3)))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(train_data, train_labels, epochs=10, batch_size=10, validation_data=(val_data, val_labels))

loss, accuracy = model.evaluate(test_data, test_labels)
print('Test accuracy:', accuracy)